In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import Dense, LSTM, Dropout
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_squared_error
# from sklearn import metrics

In [2]:
from tqdm.notebook import tqdm
import warnings

In [5]:
device_full_files = pd.DataFrame()
local = False

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
    for year in tqdm([2017, 2018, 2019, 2020, 2022, 2023]):
        if local:
            device_file = pd.read_csv('./data/device%d.zip' % year)
        else:
            device_file = pd.read_csv('https://www.accessdata.fda.gov/MAUDE/ftparea/device%d.zip' % year, sep='|', quoting=3, encoding = "ISO-8859-1")
            device_file.to_csv('./data/device%d.zip' % year, index = False)
    device_full_files = pd.concat([device_full_files, device_file], ignore_index=True)
    device_full_files.to_csv('./data/device_full_files.zip', index = False)        

  0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
# patient_filtered = patient[patient['GENERIC_NAME'].str.contains("MESH, SURGICAL, SYNTHETIC, UROGYNECOLOGIC, FOR STRESS URINARY INCONTINENCE, RETR").fillna(False)]

In [27]:
device_full_files.shape

(4684026, 31)

In [5]:
device_full_files.columns

Index(['MDR_REPORT_KEY', 'DEVICE_EVENT_KEY', 'IMPLANT_FLAG',
       'DATE_REMOVED_FLAG', 'DEVICE_SEQUENCE_NO', 'DATE_RECEIVED',
       'BRAND_NAME', 'GENERIC_NAME', 'MANUFACTURER_D_NAME',
       'MANUFACTURER_D_ADDRESS_1', 'MANUFACTURER_D_ADDRESS_2',
       'MANUFACTURER_D_CITY', 'MANUFACTURER_D_STATE_CODE',
       'MANUFACTURER_D_ZIP_CODE', 'MANUFACTURER_D_ZIP_CODE_EXT',
       'MANUFACTURER_D_COUNTRY_CODE', 'MANUFACTURER_D_POSTAL_CODE',
       'DEVICE_OPERATOR', 'EXPIRATION_DATE_OF_DEVICE', 'MODEL_NUMBER',
       'CATALOG_NUMBER', 'LOT_NUMBER', 'OTHER_ID_NUMBER',
       'DEVICE_AVAILABILITY', 'DATE_RETURNED_TO_MANUFACTURER',
       'DEVICE_REPORT_PRODUCT_CODE', 'DEVICE_AGE_TEXT',
       'DEVICE_EVALUATED_BY_MANUFACTUR', 'COMBINATION_PRODUCT_FLAG', 'UDI-DI',
       'UDI-PUBLIC'],
      dtype='object')

In [6]:
foitext_files = pd.DataFrame()
local = True
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
    for year in tqdm([2017, 2018, 2019, 2020, 2022, 2023]):
        if local:
            foitext_file = pd.read_csv('./data/foitext%d.zip' % year)
        else:
            foitext_file = pd.read_csv('https://www.accessdata.fda.gov/MAUDE/ftparea/foitext%d.zip' % year, sep='|', quoting=3, encoding = "ISO-8859-1")
            foitext_file.to_csv('data/foitext%d.zip' % year, index = False)
    foitext_files = pd.concat([foitext_files, foitext_file], ignore_index=True)
    foitext_files.to_csv('./data/foitext_files.zip', index = False)        

  0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
foitext_files.shape

(5349886, 6)

In [8]:
foitext_files.tail()

,MDR_REPORT_KEY,MDR_TEXT_KEY,TEXT_TYPE_CODE,PATIENT_SEQUENCE_NUMBER,DATE_REPORT,FOI_TEXT
5349881,17691083,344967545,D,0,NaN,IT WAS REPORTED THAT SIGNAL LOSS OVER ONE HOUR...
5349882,17691083,344967546,N,0,NaN,(B)(4).
5349883,17720251,344967659,D,0,NaN,IT WAS REPORTED THAT SIGNAL LOSS OVER ONE HOUR...
5349884,17720251,344967660,N,0,NaN,(B)(4).
5349885,16857746,344968002,N,0,NaN,"OTHER, OTHER TEXT: ADDITIONAL INFORMATION IS P..."


In [10]:
foitext_files.columns

Index(['MDR_REPORT_KEY', 'MDR_TEXT_KEY', 'TEXT_TYPE_CODE',
       'PATIENT_SEQUENCE_NUMBER', 'DATE_REPORT', 'FOI_TEXT'],
      dtype='object')

### Merge data

In [14]:
device_full_files = device_full_files[device_full_files['MDR_REPORT_KEY'].apply(type) == int] 
device_full_files = device_full_files.set_index('MDR_REPORT_KEY')

In [15]:
merged_files = foitext_files.join(device_full_files, on = 'MDR_REPORT_KEY', how = 'inner', rsuffix='_device')

In [16]:
merged_files.head()

,MDR_REPORT_KEY,MDR_TEXT_KEY,TEXT_TYPE_CODE,PATIENT_SEQUENCE_NUMBER,DATE_REPORT,FOI_TEXT,DEVICE_EVENT_KEY,IMPLANT_FLAG,DATE_REMOVED_FLAG,DEVICE_SEQUENCE_NO,...,LOT_NUMBER,OTHER_ID_NUMBER,DEVICE_AVAILABILITY,DATE_RETURNED_TO_MANUFACTURER,DEVICE_REPORT_PRODUCT_CODE,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,COMBINATION_PRODUCT_FLAG,UDI-DI,UDI-PUBLIC
59121,16146804,307178579,N,0,NaN,MEDTRONIC IS SUBMITTING THIS REPORT TO COMPLY ...,NaN,NaN,NaN,1,...,NaN,NaN,N,NaN,LWS,DA,*,N,00643169309760,00643169309760
59122,16146804,307178580,D,0,NaN,IT WAS REPORTED THAT THE RIGHT VENTRICULAR (RV...,NaN,NaN,NaN,1,...,NaN,NaN,N,NaN,LWS,DA,*,N,00643169309760,00643169309760
4244452,16146804,333665285,N,0,NaN,MEDTRONIC IS SUBMITTING THIS REPORT TO COMPLY ...,NaN,NaN,NaN,1,...,NaN,NaN,N,NaN,LWS,DA,*,N,00643169309760,00643169309760
4244453,16146804,333665286,D,0,NaN,IT WAS FURTHER REPORTED THAT THE PATIENT HAD A...,NaN,NaN,NaN,1,...,NaN,NaN,N,NaN,LWS,DA,*,N,00643169309760,00643169309760
59123,16146808,307178590,D,0,NaN,IT WAS REPORTED THAT THE PATIENT HAD A CHRONIC...,NaN,NaN,NaN,1,...,NaN,NaN,R,NaN,DSQ,DA,Y,N,NaN,NaN


In [18]:
merged_files.shape

(3244371, 36)

In [12]:
foitext_files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5349886 entries, 0 to 5349885
Data columns (total 6 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   MDR_REPORT_KEY           int64  
 1   MDR_TEXT_KEY             int64  
 2   TEXT_TYPE_CODE           object 
 3   PATIENT_SEQUENCE_NUMBER  int64  
 4   DATE_REPORT              float64
 5   FOI_TEXT                 object 
dtypes: float64(1), int64(3), object(2)
memory usage: 244.9+ MB


In [13]:
device_full_files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342013 entries, 0 to 2342012
Data columns (total 31 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   MDR_REPORT_KEY                  object
 1   DEVICE_EVENT_KEY                object
 2   IMPLANT_FLAG                    object
 3   DATE_REMOVED_FLAG               object
 4   DEVICE_SEQUENCE_NO              object
 5   DATE_RECEIVED                   object
 6   BRAND_NAME                      object
 7   GENERIC_NAME                    object
 8   MANUFACTURER_D_NAME             object
 9   MANUFACTURER_D_ADDRESS_1        object
 10  MANUFACTURER_D_ADDRESS_2        object
 11  MANUFACTURER_D_CITY             object
 12  MANUFACTURER_D_STATE_CODE       object
 13  MANUFACTURER_D_ZIP_CODE         object
 14  MANUFACTURER_D_ZIP_CODE_EXT     object
 15  MANUFACTURER_D_COUNTRY_CODE     object
 16  MANUFACTURER_D_POSTAL_CODE      object
 17  DEVICE_OPERATOR                 object
 18  EX

In [31]:
merged_files_filtered = merged_files[merged_files["GENERIC_NAME"].str.contains("STRESS URINARY INCONTINENCE")== True]
merged_files_filtered.head()

,MDR_REPORT_KEY,MDR_TEXT_KEY,TEXT_TYPE_CODE,PATIENT_SEQUENCE_NUMBER,DATE_REPORT,FOI_TEXT,DEVICE_EVENT_KEY,IMPLANT_FLAG,DATE_REMOVED_FLAG,DEVICE_SEQUENCE_NO,...,LOT_NUMBER,OTHER_ID_NUMBER,DEVICE_AVAILABILITY,DATE_RETURNED_TO_MANUFACTURER,DEVICE_REPORT_PRODUCT_CODE,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,COMBINATION_PRODUCT_FLAG,UDI-DI,UDI-PUBLIC
61009,16148538,307201920,D,0,NaN,IT WAS REPORTED TO BOSTON SCIENTIFIC CORPORATI...,NaN,NaN,NaN,1,...,0023347113,NaN,N,NaN,OTN,NaN,R,N,08714729772880,08714729772880
61010,16148538,307201921,N,0,NaN,"DATE OF EVENT WAS APPROXIMATED TO (B)(6) 2020,...",NaN,NaN,NaN,1,...,0023347113,NaN,N,NaN,OTN,NaN,R,N,08714729772880,08714729772880
63072,16149148,307215045,D,0,NaN,IT WAS REPORTED TO BOSTON SCIENTIFIC CORPORATI...,NaN,NaN,NaN,1,...,0000050202,NaN,N,NaN,OTN,NaN,R,N,08714729718987,08714729718987
63073,16149148,307215046,N,0,NaN,"DATE OF EVENT WAS APPROXIMATED TO (B)(6) 2017,...",NaN,NaN,NaN,1,...,0000050202,NaN,N,NaN,OTN,NaN,R,N,08714729718987,08714729718987
68173,16148572,307287715,D,0,NaN,I HAVE HAD THREE SURGERIES FOR BLADDER ISSUES ...,NaN,NaN,NaN,1,...,NaN,NaN,Y,NaN,OTN,NaN,NaN,N,NaN,NaN


In [32]:
merged_files_filtered.shape

(1113, 36)

In [33]:
merged_files_filtered["GENERIC_NAME"].value_counts()

GENERIC_NAME
MESH, SURGICAL, SYNTHETIC, UROGYNECOLOGIC, FOR STRESS URINARY INCONTINENCE, RETR    892
FEMALE STRESS URINARY INCONTINENCE SURGICAL MESH-SLING, SYNTHETIC POLYMER           140
MESH, SURGICAL, SYNTHETIC, UROGYNECOLOGIC, FOR STRESS URINARY INCONTINENCE, FEMA     44
MESH SURGICAL FOR STRESS URINARY INCONTINENCE MALE                                   27
MALE STRESS URINARY INCONTINENCE SURGICAL MESH-SLING, SYNTHETIC POLYMER               7
MESH, SURGICAL, SYNTHETIC, UROGYNECOLOGIC, FOR STRESS URINARY INCONTINENCE            2
STRESS URINARY INCONTINENCE, RETROPUBIC OR TRANSOBTURATOR                             1
Name: count, dtype: int64

In [34]:
merged_files_filtered["GENERIC_NAME"].count()

1113

In [28]:
"STRESS URINARY INCONTINENCE".lower()

'stress urinary incontinence'

In [36]:
# merged_files_filtered.to_csv('./data/stress_urinary_incontinence.csv')

In [37]:
patient_text = merged_files_filtered["FOI_TEXT"].dropna()

In [39]:
patient_text.shape

(1113,)